In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

from mnist import train_images, test_images, train_labels, test_labels
import numpy as np
import math
from collections import namedtuple
from functools import partial

np.seterr(all='raise')
pass

In [2]:
train_images_my = train_images().reshape((train_images().shape[0], -1)).astype(np.float) / 255
test_images_my = test_images().reshape((test_images().shape[0], -1)).astype(np.float) / 255
train_labels_my = np.identity(10)[train_labels()]
test_labels_my = np.identity(10)[test_labels()]

In [9]:
class nnClassifier():
    def __init__(self, h, ll):
        self.h = h
        self.ll = ll

    def fit(self, x, y):
        n = x[0].shape[0]
        m = y[0].shape[0]
        l = x.shape[0]
        sigma = lambda x : (1 + 2.7182818284590452354 ** (-x)) ** (-1)
        
        w0 = (np.random.random((n, self.h)) - 0.5) / n
        w1 = (np.random.random((self.h, m)) - 0.5) / m
        q = l / 10.0
        EPS = 1e-5

        while True:
            i = np.random.randint(l)
            u = sigma(x[i].dot(w0))
            a = sigma(u.dot(w1))
            eps1 = a - y[i]
            q_i = np.sum(eps1 * eps1)
            
            eps0 = w1.dot(eps1 * a * (1 - a))
            
            w1 -= self.ll * np.outer(u,    eps1 * a * (1 - a))
            w0 -= self.ll * np.outer(x[i], eps0 * u * (1 - u))
            new_q = (l - 1) / l * q + q_i / l
            
            if abs(new_q - q) < EPS:
                break
            if np.random.randint(30000) == 0:
                print(q)
            q = new_q
            
        self.w0 = w0
        self.w1 = w1

    def predict(self, point):
        sigma = lambda x : (1 + 2.7182818284590452354 ** (-x)) ** (-1)
        u = sigma(point.dot(self.w0))
        a = sigma(u.dot(self.w1))
        return a

    def __str__(self):
        return "nn classifier with h = {} and ll = {}".format(str(self.h), str(self.ll))

In [10]:
nn = nnClassifier(28, 1)

In [11]:
nn.fit(train_images_my, train_labels_my)

5951.70185002
5894.15594768
5771.19422324
5720.82325635
5560.82773124
5547.31337923
5535.12307752
5481.05400769
5480.04929493
5320.48787608
5284.60837049
5074.4403274
5007.64920069
4970.39912771
4916.18946055
4860.05965537
4858.43998681
4741.72742445
4738.48847246
4596.61056569
4508.74953979
4435.41158906
4422.936525
4417.2643556
4359.55967982
4150.7506844
4129.70514538
4111.57491435
4060.97526364
4031.03596705
4020.10010585
3853.47338993
3817.93080318
3798.76263787
3688.46213295
3645.56042954
3535.64040417
3434.98352622
3303.87854154
3228.4905344
3216.67480671
3193.0133313
3140.86772697
2894.37471758
2801.07737231
2673.10233561
2657.15679468
2579.23105893
2563.4182264
2529.72107638
2474.71854392
2442.47219976
2439.21794776
2394.66889361
2373.80794438
2253.49241362
2243.78641949
2187.00454761
2129.49369675
2123.64605781
2120.88716656
2051.56485382
2012.21726049
1999.34756908
1928.07686278
1924.83429501
1850.84555806
1813.1406857
1795.10071706
1752.38773226
1706.13557684
1689.0762168
16

In [14]:
sigma = lambda x : (1 + 2.7182818284590452354 ** (-x)) ** (-1)
nn.predict(train_images_my[0])

array([  4.44398301e-07,   2.39389943e-04,   2.74071845e-06,
         2.36621272e-02,   2.19249751e-12,   9.99970663e-01,
         1.37319344e-10,   3.87139930e-06,   5.14725377e-07,
         2.28401836e-09])

In [16]:
np.argmax(nn.predict(train_images_my[0]))

5

In [17]:
nn.predict(test_images_my[0])

array([  1.10012699e-09,   4.02459985e-08,   3.19674996e-08,
         2.87832275e-08,   5.95005219e-11,   9.70112226e-11,
         1.71780739e-16,   9.99999925e-01,   2.55740223e-13,
         2.59845341e-09])

In [18]:
np.argmax(nn.predict(test_images_my[0]))

7

In [20]:
np.argmax(test_labels_my[0])

7